# Transfer Learning with Pytorch for image classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torchinfo import summary
import numpy as np

## Download the data

In [2]:
ls data/hymenoptera_data/

train/ val/


In [3]:
#There are 124 images of ants for training and 70 images for validation.

!ls data/hymenoptera_data/train/ants/ | wc -l

     124


In [4]:
!ls data/hymenoptera_data/val/ants/ | wc -l

      70


In [5]:
# There are 121 images of bees for training and 83 images for validation.
                                                
!ls data/hymenoptera_data/train/bees/ | wc -l

     121


In [6]:
ls data/hymenoptera_data/val/bees/ | wc -l

      83


## Data Preprocessing

In [7]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456,0.406], 
        [0.229, 0.224, 0.225])])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406], 
        [0.229, 0.224, 0.225])])


In [8]:
train_dataset = datasets.ImageFolder(
          root='data/hymenoptera_data/train',
          transform=train_transforms)

val_dataset = datasets.ImageFolder(
            root='data/hymenoptera_data/val',
            transform=val_transforms)

In [9]:
train_loader = torch.utils.data.DataLoader(
            train_dataset, 
            batch_size=4,
            shuffle=True, 
            num_workers=4)

val_loader = torch.utils.data.DataLoader(
            val_dataset, 
            batch_size=4,
            shuffle=True, 
            num_workers=4)

# Model Design

In [10]:
from torchvision.models import ResNet18_Weights
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

print(model.fc)
# out: Linear(in_features=512, out_features=1000, bias=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
print(model.fc)
# out: Linear(in_features=512, out_features=2, bias=True)

Linear(in_features=512, out_features=1000, bias=True)
Linear(in_features=512, out_features=2, bias=True)


In [11]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.we

## Training and Validation

In [12]:
from torch.optim.lr_scheduler import StepLR

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

model = model.to(device)
loss_fn = nn.CrossEntropyLoss() 
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 
exp_lr_scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
num_epochs=30

for epoch in range(num_epochs):

    # Training phase
    
    model.train() 
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
                
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()/inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)/inputs.size(0)

  
    exp_lr_scheduler.step() 
    train_loss = running_loss/len(train_loader)
    train_acc = running_corrects/len(train_loader)

    
    # Validation phase
    
    model.eval() 
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        running_loss += loss.item() / inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)/inputs.size(0)

    val_loss = running_loss/len(val_loader)
    val_acc = running_corrects.double()/len(val_loader)
    
    print("Train: Loss = {:.4f} Acc = {:.4f}; Val: Loss = {:.4f} Acc = {:.4f}".format(train_loss, train_acc, val_loss, val_acc))

Train: Loss = 0.2001 Acc = 0.6844; Val: Loss = 0.1505 Acc = 0.7756
Train: Loss = 0.1560 Acc = 0.7295; Val: Loss = 0.0652 Acc = 0.9167
Train: Loss = 0.1105 Acc = 0.8197; Val: Loss = 0.0657 Acc = 0.9167
Train: Loss = 0.1354 Acc = 0.7787; Val: Loss = 0.0766 Acc = 0.9167
Train: Loss = 0.1937 Acc = 0.7623; Val: Loss = 0.0827 Acc = 0.8974
Train: Loss = 0.1522 Acc = 0.7664; Val: Loss = 0.1850 Acc = 0.7244
Train: Loss = 0.1592 Acc = 0.7828; Val: Loss = 0.1779 Acc = 0.8013
Train: Loss = 0.1093 Acc = 0.8484; Val: Loss = 0.0678 Acc = 0.9038
Train: Loss = 0.0901 Acc = 0.8648; Val: Loss = 0.0606 Acc = 0.9167
Train: Loss = 0.0763 Acc = 0.8689; Val: Loss = 0.0575 Acc = 0.9167
Train: Loss = 0.0695 Acc = 0.8730; Val: Loss = 0.0558 Acc = 0.9423
Train: Loss = 0.0911 Acc = 0.8279; Val: Loss = 0.0553 Acc = 0.9295
Train: Loss = 0.0754 Acc = 0.8689; Val: Loss = 0.0523 Acc = 0.9231
Train: Loss = 0.0793 Acc = 0.8648; Val: Loss = 0.0584 Acc = 0.9295
Train: Loss = 0.0656 Acc = 0.8811; Val: Loss = 0.0511 Acc = 0.

### Saving the model 

In [14]:
torch.save(model.state_dict(), "./modified_resnet18.pt")